#**DETECT FAKES NEWS USING NEURAL NETWORKS**

#1.Data preparation

In [1]:
import pandas as pd
import tensorflow as tf



In [2]:
from tensorflow.keras.layers import TextVectorization
import re
import string

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split


In [4]:
fake=pd.read_csv('/content/drive/MyDrive/Data_set/fake.csv')
fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [5]:
true=pd.read_csv('/content/drive/MyDrive/Data_set/true.csv')
true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [6]:
true['type']=0
fake['type']=1

In [7]:
news=pd.concat([fake,true])
news

,title,text,subject,date,type
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [8]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   type     44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


#2.Preprocessing

In [9]:
news=news.sample(frac=1)


In [10]:
news.reset_index(inplace=True)

In [11]:
news

,index,title,text,subject,date,type
0,16782,FOX NEWS BIGGIE Promoting Mass Immigration To ...,Has anyone else out there noticed the total bi...,Government News,"Feb 6, 2016",1
1,14785,BUDGET DEAL Quadruples Number Of Unskilled Gue...,Speaker of the House Paul Ryan is officially a...,politics,"Dec 17, 2015",1
2,13489,French defense minister hopeful for fighter je...,PARIS (Reuters) - France is hoping to strike d...,worldnews,"November 30, 2017",0
3,21530,WHILE OBAMA VACATIONS AND HANDS OUT A BILLION ...,Hey Barry tell us again about what a great dea...,left-news,"Jul 25, 2015",1
4,11923,Britain joins U.S. in blaming North Korea for ...,LONDON (Reuters) - Britain said North Korea wa...,worldnews,"December 19, 2017",0
...,...,...,...,...,...,...
44893,13029,"Greece, Turkey seek closer ties with Erdogan v...",ATHENS/ANKARA (Reuters) - Turkish President Ta...,worldnews,"December 6, 2017",0
44894,15019,Former intelligence officials say Trump is bei...,WASHINGTON (Reuters) - Two former top U.S. int...,worldnews,"November 12, 2017",0
44895,16206,Xi ally Li Qiang named Shanghai party boss: Xi...,"BEIJING (Reuters) - Li Qiang, an ally of Chine...",worldnews,"October 29, 2017",0
44896,5053,Even This Fox News Host Felt The Need To Shut...,"Usually, Donald Trump spokesperson Katrina Pie...",News,"August 14, 2016",1


In [12]:
news.drop(columns=['index'],inplace=True)

In [13]:
news

,title,text,subject,date,type
0,FOX NEWS BIGGIE Promoting Mass Immigration To ...,Has anyone else out there noticed the total bi...,Government News,"Feb 6, 2016",1
1,BUDGET DEAL Quadruples Number Of Unskilled Gue...,Speaker of the House Paul Ryan is officially a...,politics,"Dec 17, 2015",1
2,French defense minister hopeful for fighter je...,PARIS (Reuters) - France is hoping to strike d...,worldnews,"November 30, 2017",0
3,WHILE OBAMA VACATIONS AND HANDS OUT A BILLION ...,Hey Barry tell us again about what a great dea...,left-news,"Jul 25, 2015",1
4,Britain joins U.S. in blaming North Korea for ...,LONDON (Reuters) - Britain said North Korea wa...,worldnews,"December 19, 2017",0
...,...,...,...,...,...
44893,"Greece, Turkey seek closer ties with Erdogan v...",ATHENS/ANKARA (Reuters) - Turkish President Ta...,worldnews,"December 6, 2017",0
44894,Former intelligence officials say Trump is bei...,WASHINGTON (Reuters) - Two former top U.S. int...,worldnews,"November 12, 2017",0
44895,Xi ally Li Qiang named Shanghai party boss: Xi...,"BEIJING (Reuters) - Li Qiang, an ally of Chine...",worldnews,"October 29, 2017",0
44896,Even This Fox News Host Felt The Need To Shut...,"Usually, Donald Trump spokesperson Katrina Pie...",News,"August 14, 2016",1


In [14]:
news.tail(20)

,title,text,subject,date,type
44878,U.S. Democrats' infighting over trade pact put...,NEW YORK/WASHINGTON (Reuters) - Hillary Clinto...,politicsNews,"July 8, 2016",0
44879,IS MEGYN KELLY Planning To Leave FOX News? [VI...,Did Donald Trump convince Megyn Kelly s loyal ...,politics,"Apr 5, 2016",1
44880,BREAKING: FBI Carried Out Warrant On Republic...,The LA Times is reporting that the Federal Bur...,News,"August 9, 2017",1
44881,Red Cross Sorry For Blatantly Racist Pool Saf...,Margaret Sawyer was shocked when she encounter...,News,"June 26, 2016",1
44882,N.Y. power company sends crew to aid Puerto Ri...,(Reuters) - The New York Power Authority (NYPA...,worldnews,"September 22, 2017",0
44883,Planned Parenthood’s Latest Announcement Will...,You know what scares Republicans more than bla...,News,"August 5, 2016",1
44884,“HILL”-ARIOUS VIDEO: HILLARY TAKES BREAK From ...,This video brilliantly dispels every one of Hi...,left-news,"Aug 26, 2016",1
44885,"U.S., backed by China, proposes tough N.Korea ...",UNITED NATIONS (Reuters) - The United States p...,politicsNews,"February 26, 2016",0
44886,Italian government wins all five confidence vo...,ROME (Reuters) - The Italian government won al...,worldnews,"October 25, 2017",0
44887,GA TOWN MANDATES GUN OWNERSHIP: Here’s What Ha...,The results of this town s gun ownership manda...,Government News,"Jan 11, 2016",1


In [15]:
news.insert(0,'news',news['title']+news['text'])


In [16]:
news.drop(columns=['title','text'],inplace=True)

In [17]:
news

,news,subject,date,type
0,FOX NEWS BIGGIE Promoting Mass Immigration To ...,Government News,"Feb 6, 2016",1
1,BUDGET DEAL Quadruples Number Of Unskilled Gue...,politics,"Dec 17, 2015",1
2,French defense minister hopeful for fighter je...,worldnews,"November 30, 2017",0
3,WHILE OBAMA VACATIONS AND HANDS OUT A BILLION ...,left-news,"Jul 25, 2015",1
4,Britain joins U.S. in blaming North Korea for ...,worldnews,"December 19, 2017",0
...,...,...,...,...
44893,"Greece, Turkey seek closer ties with Erdogan v...",worldnews,"December 6, 2017",0
44894,Former intelligence officials say Trump is bei...,worldnews,"November 12, 2017",0
44895,Xi ally Li Qiang named Shanghai party boss: Xi...,worldnews,"October 29, 2017",0
44896,Even This Fox News Host Felt The Need To Shut...,News,"August 14, 2016",1


In [18]:
X=news['news']
y=news['type']
X=np.array(X)
y=np.array(y)

In [19]:
X

array(['FOX NEWS BIGGIE Promoting Mass Immigration To The U.S. Via FOX NewsHas anyone else out there noticed the total bias with promotion of certain candidates running for president? I ve noticed who they re supporting like never before. I should say I ve noticed who they detest also Trump No matter which candidate you support, this isn t fair to the American people who really want the truth in the media to come back. FOX is now no better than the liberal leaning CNN and MSNBC. Watch with caution after you read this excellent piece about the connections between big business, politics and the push for mass immigration to America. Yes folks, it s all about cheap labor and money Mr. Rupert s most visible investment is Fox s princess Megyn Kelly, who just coincidentally signed a book deal with another of Ruperts  business interests, Harper Collins books.Murdoch is a long time supporter of Senator Marco Rubio. The Fox News head even structured media coverage on behalf of Rubio during the G

In [20]:
y


array([1, 1, 0, ..., 0, 1, 0])

In [21]:
X_train_full,X_test,y_train_full,y_test=train_test_split(X,y,test_size=0.15)


In [22]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train_full,y_train_full,test_size=0.15/0.85)



In [23]:
X_train


array(['U.S. approves license for Ukraine to buy small arms from U.S. firmsWASHINGTON (Reuters) - The U.S. State Department has approved an export license for Ukraine to buy certain light weapons and small arms from U.S. manufacturers, spokeswoman Heather Nauert said on Wednesday. Department records show Ukraine has bought small amounts of those types of weapons for several years, both before and after the 2014 Russian annexation of Ukraine s Crimean peninsula. The department notified Congress of the decision on Dec. 13, Nauert said, adding that the U.S. government was not selling the weapons directly to the Kiev government but was allowing Ukraine to buy from U.S. manufacturers.   Under the previous two administrations, the U.S. government has approved export licenses to Ukraine, so this is nothing new,  Nauert said. The license covers weapons in categories such as semi-automatic and automatic firearms up to, and including, .50 caliber weapons. It also includes combat shotguns, silenc

In [24]:
y_train

array([0, 1, 0, ..., 1, 0, 0])

In [25]:
print(len(y_train),len(y_valid),len(y_test))

31428 6735 6735


#3.Model building

In [26]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

In [27]:
news['news']

0        FOX NEWS BIGGIE Promoting Mass Immigration To ...
1        BUDGET DEAL Quadruples Number Of Unskilled Gue...
2        French defense minister hopeful for fighter je...
3        WHILE OBAMA VACATIONS AND HANDS OUT A BILLION ...
4        Britain joins U.S. in blaming North Korea for ...
                               ...                        
44893    Greece, Turkey seek closer ties with Erdogan v...
44894    Former intelligence officials say Trump is bei...
44895    Xi ally Li Qiang named Shanghai party boss: Xi...
44896     Even This Fox News Host Felt The Need To Shut...
44897    South African reshuffle irks ANC allies, Zuma ...
Name: news, Length: 44898, dtype: object

In [28]:
vocab_size=10000
sequence_length=150
vectorize=tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length


)

In [29]:
vectorize.adapt(X)

In [30]:
model=tf.keras.models.Sequential([
    vectorize,
    tf.keras.layers.Embedding(len(vectorize.get_vocabulary()),64,mask_zero=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation='sigmoid')

    ])

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),loss='binary_crossentropy',metrics=['accuracy'])

In [32]:
callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)

In [33]:
model.fit(X_train,y_train,validation_data=(X_valid,y_valid),epochs=10,callbacks=[callbacks],batch_size=32)


Epoch 1/10
983/983 [==============================] - 84s 72ms/step - loss: 0.2155 - accuracy: 0.9154 - val_loss: 0.0337 - val_accuracy: 0.9936
Epoch 2/10
983/983 [==============================] - 30s 30ms/step - loss: 0.0235 - accuracy: 0.9948 - val_loss: 0.0147 - val_accuracy: 0.9966
Epoch 3/10
983/983 [==============================] - 27s 27ms/step - loss: 0.0082 - accuracy: 0.9980 - val_loss: 0.0119 - val_accuracy: 0.9963
Epoch 4/10
983/983 [==============================] - 25s 26ms/step - loss: 0.0035 - accuracy: 0.9995 - val_loss: 0.0148 - val_accuracy: 0.9963
Epoch 5/10
983/983 [==============================] - 24s 25ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.0131 - val_accuracy: 0.9964
Epoch 6/10
983/983 [==============================] - 25s 26ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0160 - val_accuracy: 0.9960
Epoch 7/10
983/983 [==============================] - 25s 26ms/step - loss: 4.2443e-04 - accuracy: 0.9999 - val_loss: 0.0207 - val_accur

In [34]:
model.evaluate(X_test,y_test)

211/211 [==============================] - 3s 15ms/step - loss: 0.0189 - accuracy: 0.9960


[0.018879523500800133, 0.9959911108016968]

In [35]:
text=['If 1+1=0 then 2/0=1']


In [36]:
result=model.predict(np.array(text))
if result>0.5:
  print('fakes news: ',result)
else:
  print('good news; ',result)

1/1 [==============================] - 2s 2s/step
fakes news:  [[0.8941148]]
